In [219]:
from pyspark import SparkContext, SparkConf

In [220]:
sc

<SparkContext master=yarn appName=PySparkShell>

In [221]:
rdd = sc.textFile("/data/lsml/sga/clickstream.csv")

In [222]:
#rdd.take(10)

In [223]:
rdd.count()

5000001

In [224]:
rdd1 = rdd.map(lambda x: x.split('\t'))

In [225]:
#rdd1.take(10)

In [226]:
header = rdd1.first()
rdd2 = rdd1.filter(lambda row: row != header) # filter out the header

In [227]:
#rdd2.take(10)

In [228]:
#rdd2.count()

In [229]:
rdd_error = rdd2.filter(lambda x: "error" in x[2])

In [230]:
#rdd_error.take(10)

In [231]:
rdd_error1 = rdd_error.map(lambda x : ((x[0],x[1]), int(x[4])))

In [232]:
#rdd_error1.take(10)

In [233]:
rdd_error2 = rdd_error1.reduceByKey(lambda x,y : x if x<y else y)

In [234]:
#rdd_error2.sortByKey().take(10)

In [235]:
pages1 = rdd2.filter(lambda x : x[2] == 'page')

In [236]:
#pages1.take(10)

In [237]:
pages2 = pages1.map(lambda x: ((x[0],x[1]), (x[3], int(x[4]))))

In [238]:
#pages2.take(10)

In [239]:
merged = pages2.leftOuterJoin(rdd_error2)

In [240]:
#merged.take(10)

In [241]:
merged2 = merged1.map((lambda x: ((x[0][0], x[0][1]), (x[1][0][0], x[1][0][1], x[1][1]))))

In [242]:
#merged2.take(10)

In [243]:
merged3 = merged2.filter(lambda x: True if (x[1][2] is None or x[1][1]<x[1][2]) else False )

In [244]:
#merged3.take(15)

In [245]:
merged4 = merged3.map(lambda x: ((x[0][0], x[0][1], x[1][1]), x[1][0]))

In [246]:
#merged4.take(15)

In [247]:
merged5 = merged4.sortByKey()

In [248]:
#merged5.take(15)

In [249]:
merged6 = merged5.map(lambda x: ((x[0][0], x[0][1]), x[1]))

In [250]:
#merged6.take(15)

In [251]:
merged7 = merged6.foldByKey('', (lambda x, y: x + '-' + y))

In [252]:
#merged7.take(10)

In [253]:
#merged7.lookup(('0', '1013'))

In [254]:
merged8 = merged7.map(lambda x:  (x[1][1:], 1))

In [255]:
#merged8.take(10)

In [256]:
merged9 = merged8.reduceByKey(lambda x, y : x+y)

In [257]:
answers = merged9.sortBy((lambda x: x[1]), False)

In [258]:
answers.take(30)

[('main', 39256),
 ('main-tariffs', 6536),
 ('main-news', 6274),
 ('main-archive', 5849),
 ('main-family', 4863),
 ('main-digital', 4224),
 ('main-bonus', 3495),
 ('main-tariffs-news', 1189),
 ('main-news-tariffs', 1131),
 ('main-tariffs-archive', 1037),
 ('main-news-archive', 1002),
 ('main-archive-news', 998),
 ('main-archive-tariffs', 997),
 ('main-family-tariffs', 922),
 ('main-news-family', 919),
 ('main-tariffs-family', 918),
 ('main-family-news', 880),
 ('main-archive-family', 818),
 ('main-news-digital', 797),
 ('main-family-archive', 775),
 ('main-tariffs-main', 761),
 ('main-tariffs-digital', 751),
 ('main-digital-news', 748),
 ('main-digital-tariffs', 722),
 ('main-archive-digital', 720),
 ('main-spravka', 708),
 ('main-news-main', 686),
 ('main-digital-archive', 683),
 ('main-tariffs-bonus', 667),
 ('main-news-bonus', 617)]